In [217]:
import pandas as pd
import numpy as np
from keras.optimizers import Adam
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.preprocessing import MultiLabelBinarizer, OneHotEncoder
from sklearn.model_selection import train_test_split

from tensorflow import keras

from keras.models import Sequential
from keras.layers import Dense, Dropout
# import keras_tuner

In [218]:
merged_data_path = '../data/merged_data.jsonl'
data = pd.read_json(merged_data_path, lines=True)

# data = data.drop(
#     columns=["release_date", "key", "loudness",
#              "explicit", "popularity", "duration_ms", "danceability", "energy", "speechiness",
#              "acousticness", "instrumentalness", "liveness", "valence", "tempo"])

data = data.drop(
    columns=["release_date", "name"])

data.head(5)

,timestamp,user_id,track_id,session_id,track_name,popularity,duration_ms,explicit,id_artist,danceability,...,city,street,favourite_genres,premium_user,genres,skipped,number_of_matching_genres,month,day_of_week,hour_of_day
0,2023-01-04 00:12:59.000,101,0NPjiwqT1xrA3ck05xKoA8,124,"Anton skaffar sig hund, del 4",21,159812,0,1c6OwPjqCGGUg770n3zhbq,0.667,...,Szczecin,pl. Floriana 55/22,"[permanent wave, mandopop, funk]",False,[barnsagor],False,0,1,2,0
1,2023-02-04 01:20:47.302,926,0NPjiwqT1xrA3ck05xKoA8,11284,"Anton skaffar sig hund, del 4",21,159812,0,1c6OwPjqCGGUg770n3zhbq,0.667,...,Szczecin,pl. Lawendowa 35/83,"[alternative metal, rock, c-pop]",False,[barnsagor],False,0,2,5,1
2,2023-01-28 02:30:48.635,926,1hviQqMhM4NyY4O6CWZABO,11281,"Det finns väl ingen med kniv här i stan, del 3",20,113858,0,1c6OwPjqCGGUg770n3zhbq,0.676,...,Szczecin,pl. Lawendowa 35/83,"[alternative metal, rock, c-pop]",False,[barnsagor],True,0,1,5,2
3,2023-02-14 21:59:20.619,940,0NPjiwqT1xrA3ck05xKoA8,11473,"Anton skaffar sig hund, del 4",21,159812,0,1c6OwPjqCGGUg770n3zhbq,0.667,...,Wrocław,ulica Witosa 13,"[ranchera, pop, latin alternative]",True,[barnsagor],True,0,2,1,21
4,2023-02-06 18:41:13.873,1095,0NPjiwqT1xrA3ck05xKoA8,13459,"Anton skaffar sig hund, del 4",21,159812,0,1c6OwPjqCGGUg770n3zhbq,0.667,...,Wrocław,pl. Orzechowa 97/37,"[singer-songwriter, pop rock, post-teen pop]",True,[barnsagor],False,0,2,0,18


In [219]:
unique_favourite_genres = set()
for genres in data['favourite_genres']:
  for genre in genres:
    unique_favourite_genres.add(genre)

unique_genres = set()
for genres in data['genres']:
  for genre in genres:
    unique_genres.add(genre)

# show all common genres between all favourite_genres and genres
print("unique_favourite_genres", len(unique_favourite_genres))
print("unique_genres", len(unique_genres))
common_genres = unique_favourite_genres.intersection(unique_genres)
print("common genres", common_genres)

# remove genres that are not in favourite_genres
# data['genres'] = data['genres'].apply(
#     lambda x: [genre for genre in x if genre in unique_favourite_genres])

data.head()

# TODO remove ??? moze niepotrzebne

unique_favourite_genres 50
unique_genres 2875
common genres {'post-teen pop', 'mpb', 'blues rock', 'soul', 'new wave', 'argentine rock', 'alternative rock', 'vocal jazz', 'adult standards', 'tropical', 'pop', 'latin rock', 'latin', 'motown', 'metal', 'mandopop', 'hard rock', 'lounge', 'turkish pop', 'roots rock', 'hoerspiel', 'new romantic', 'new wave pop', 'singer-songwriter', 'folk rock', 'pop rock', 'filmi', 'classic rock', 'quiet storm', 'permanent wave', 'mellow gold', 'j-pop', 'psychedelic rock', 'rock en espanol', 'dance pop', 'c-pop', 'latin pop', 'funk', 'rock', 'country rock', 'soft rock', 'folk', 'album rock', 'latin alternative', 'europop', 'ranchera', 'brill building pop', 'art rock', 'regional mexican', 'alternative metal'}


,timestamp,user_id,track_id,session_id,track_name,popularity,duration_ms,explicit,id_artist,danceability,...,city,street,favourite_genres,premium_user,genres,skipped,number_of_matching_genres,month,day_of_week,hour_of_day
0,2023-01-04 00:12:59.000,101,0NPjiwqT1xrA3ck05xKoA8,124,"Anton skaffar sig hund, del 4",21,159812,0,1c6OwPjqCGGUg770n3zhbq,0.667,...,Szczecin,pl. Floriana 55/22,"[permanent wave, mandopop, funk]",False,[barnsagor],False,0,1,2,0
1,2023-02-04 01:20:47.302,926,0NPjiwqT1xrA3ck05xKoA8,11284,"Anton skaffar sig hund, del 4",21,159812,0,1c6OwPjqCGGUg770n3zhbq,0.667,...,Szczecin,pl. Lawendowa 35/83,"[alternative metal, rock, c-pop]",False,[barnsagor],False,0,2,5,1
2,2023-01-28 02:30:48.635,926,1hviQqMhM4NyY4O6CWZABO,11281,"Det finns väl ingen med kniv här i stan, del 3",20,113858,0,1c6OwPjqCGGUg770n3zhbq,0.676,...,Szczecin,pl. Lawendowa 35/83,"[alternative metal, rock, c-pop]",False,[barnsagor],True,0,1,5,2
3,2023-02-14 21:59:20.619,940,0NPjiwqT1xrA3ck05xKoA8,11473,"Anton skaffar sig hund, del 4",21,159812,0,1c6OwPjqCGGUg770n3zhbq,0.667,...,Wrocław,ulica Witosa 13,"[ranchera, pop, latin alternative]",True,[barnsagor],True,0,2,1,21
4,2023-02-06 18:41:13.873,1095,0NPjiwqT1xrA3ck05xKoA8,13459,"Anton skaffar sig hund, del 4",21,159812,0,1c6OwPjqCGGUg770n3zhbq,0.667,...,Wrocław,pl. Orzechowa 97/37,"[singer-songwriter, pop rock, post-teen pop]",True,[barnsagor],False,0,2,0,18


In [220]:
# balance set

skipped_data = data[data["skipped"] == 1]
not_skipped_data = data[data["skipped"] == 0]

# Split the dataset into subsets based on class labels
print("skipped_data", skipped_data.shape)
print("not_skipped_data", not_skipped_data.shape)

# Calculate the minimum number of samples among all classes
min_samples = min(len(skipped_data), len(not_skipped_data))

# Randomly select samples from each class subset to match the minimum number of samples
skipped_data_balanced = skipped_data.sample(n=min_samples, random_state=42)
not_skipped_data_balanced = not_skipped_data.sample(n=min_samples, random_state=42)

# Merge the balanced subsets to create the final balanced dataset
balanced_data = pd.concat([skipped_data_balanced, not_skipped_data_balanced], axis=0)

# Shuffle the balanced dataset
balanced_data = balanced_data.sample(frac=1, random_state=42).reset_index(drop=True)
balanced_data.head()

skipped_data (457145, 32)
not_skipped_data (819365, 32)


,timestamp,user_id,track_id,session_id,track_name,popularity,duration_ms,explicit,id_artist,danceability,...,city,street,favourite_genres,premium_user,genres,skipped,number_of_matching_genres,month,day_of_week,hour_of_day
0,2023-03-04 15:30:45.519,816,7H6ev70Weq6DdpZyyTmUXk,9799,Say My Name,77,271333,0,1Y8cdNmUJH7yBTd9yOvr5i,0.713,...,Poznań,pl. Żurawia 379,"[europop, filmi, j-pop]",True,"[dance pop, girl group, hip pop, neo soul, pop...",True,0,3,5,15
1,2023-03-15 02:08:12.302,2064,726FgclkT72fRnkuz6bmD2,26629,Living Loving Maid (She's Just a Woman) - Rema...,38,159301,0,36QJpDe2go2KgaRleHCDTp,0.475,...,Warszawa,ulica Mickiewicza 74,"[hoerspiel, rock, permanent wave]",False,"[album rock, classic rock, hard rock, rock]",False,1,3,2,2
2,2023-03-09 12:47:03.014,3145,2CqqyQ1jH8TOGuYRX3XqBY,40984,İyileşmiyor,39,217840,0,6rGDR3wmkz5DKMrZxpO1mZ,0.630,...,Radom,plac Miarki 04/23,"[album rock, j-pop, folk]",True,"[turkish alternative, turkish alternative rock...",True,0,3,3,12
3,2023-03-06 17:32:56.113,2970,380FbszCi2WPjFUPfBL75n,38603,Bir Kulunu Çok Sevdim,29,285427,0,4cMwyqmHCwJjRZ3frIVHTr,0.523,...,Kraków,ul. Myśliwska 677,"[tropical, country rock, new wave pop]",True,[arabesk],False,0,3,0,17
4,2023-01-27 13:31:54.071,4388,5bx4stZRInFwFz40fHDa7h,57842,Hiphop N RnB,62,165283,1,6rYEqmajzlhGVaayOJ2bpJ,0.852,...,Szczecin,ul. Jaworowa 61,"[turkish pop, soft rock, new romantic]",False,"[swedish gangsta rap, swedish hip hop, swedish...",False,0,1,4,13


In [221]:
def normalize(data):
  return (data - np.min(data)) / (np.max(data) - np.min(data))

# Combine genres and favourite_genres
all_genres = list(balanced_data['favourite_genres'] + balanced_data['genres'])

# One-hot encode the genres
mlb = MultiLabelBinarizer()
mlb.fit(all_genres)

encoded_all_genres = mlb.fit_transform(all_genres)

# TODO moze trzeba inaczej enkodowac?

# Split encoded_genres into genres and favourite_genres
encoded_favourite_genres = encoded_all_genres[:, :len(balanced_data['favourite_genres'][0])]
encoded_genres = encoded_all_genres[:, len(balanced_data['favourite_genres'][0]):]

encoded_favourite_genres = mlb.transform(balanced_data['favourite_genres'])
encoded_genres = mlb.transform(balanced_data['genres'])
# encoded_common_genres = mlb.transform(data['common_genres'])

timestamp_normalized = balanced_data['timestamp'].values.reshape(-1, 1)
timestamp_normalized = normalize(timestamp_normalized)

popularity_normalized = balanced_data['popularity'].values.reshape(-1, 1)
popularity_normalized = normalize(popularity_normalized)

tempo_normalized = balanced_data['tempo'].values.reshape(-1, 1)
tempo_normalized = normalize(tempo_normalized)

duration_ms_normalized = balanced_data['duration_ms'].values.reshape(-1, 1)
duration_ms_normalized = normalize(duration_ms_normalized)

danceability_normalized = balanced_data['danceability'].values.reshape(-1, 1)
danceability_normalized = normalize(danceability_normalized)

energy_normalized = balanced_data['energy'].values.reshape(-1, 1)
energy_normalized = normalize(energy_normalized)

number_of_matching_genres_normalized = balanced_data['number_of_matching_genres'].values.reshape(-1, 1)
number_of_matching_genres_normalized = normalize(number_of_matching_genres_normalized)

hour_of_day_normalized = balanced_data['hour_of_day'].values.reshape(-1, 1)
hour_of_day_normalized = normalize(hour_of_day_normalized)

day_of_week_normalized = balanced_data['day_of_week'].values.reshape(-1, 1)
day_of_week_normalized = normalize(day_of_week_normalized)

premium_user_normalized = balanced_data['premium_user'].astype(int).values.reshape(-1, 1)
premium_user_normalized = normalize(premium_user_normalized)

# create data frame from data genres, data favourite_genres, encoded genres, encoded favourite_genres
df = pd.DataFrame(
    data={
          'genres': balanced_data['genres'],
          'favourite_genres': balanced_data['favourite_genres'],
          'skipped': balanced_data['skipped']})

df.head(50)

,genres,favourite_genres,skipped
0,"[dance pop, girl group, hip pop, neo soul, pop...","[europop, filmi, j-pop]",True
1,"[album rock, classic rock, hard rock, rock]","[hoerspiel, rock, permanent wave]",False
2,"[turkish alternative, turkish alternative rock...","[album rock, j-pop, folk]",True
3,[arabesk],"[tropical, country rock, new wave pop]",False
4,"[swedish gangsta rap, swedish hip hop, swedish...","[turkish pop, soft rock, new romantic]",False
5,"[album rock, classic rock, mellow gold, new wa...","[new wave, post-teen pop, lounge]",False
6,[austropop],"[country rock, folk rock, dance pop]",False
7,"[new wave, permanent wave, rock]","[latin, art rock, latin alternative]",True
8,"[adult standards, brill building pop, british ...","[soft rock, new wave pop, new romantic]",False
9,"[acid rock, album rock, blues rock, classic ro...","[singer-songwriter, pop, roots rock]",False


In [222]:
# rearrange X in such way, that it containts equal number of skipped=True and skipped=False


In [ ]:
# TODO pewnie trzeba zbalansowac set najpierw 

# # create pandas dataframe from encoded_genres, encoded_favourite_genres and skipped
# new_data = np.concatenate([encoded_genres, encoded_favourite_genres, data['skipped'].astype(int).values], axis=1)

# skipped_data = new_data[new_data["skipped"] == 1]
# not_skipped_data = new_data[new_data["skipped"] == 0]

# # Split the dataset into subsets based on class labels
# print("skipped_data", skipped_data.shape)
# print("not_skipped_data", not_skipped_data.shape)

# # Calculate the minimum number of samples among all classes
# min_samples = min(len(skipped_data), len(not_skipped_data))

# # Randomly select samples from each class subset to match the minimum number of samples
# skipped_data_balanced = skipped_data.sample(n=min_samples, random_state=42)
# not_skipped_data_balanced = not_skipped_data.sample(n=min_samples, random_state=42)

# # Merge the balanced subsets to create the final balanced dataset
# balanced_data = pd.concat([skipped_data_balanced, not_skipped_data_balanced], axis=0)

# # Shuffle the balanced dataset
# balanced_data = balanced_data.sample(frac=1, random_state=42).reset_index(drop=True)
# print(balanced_data.head())


# Concatenate the one-hot encoded columns
X = np.concatenate([encoded_genres, encoded_favourite_genres, premium_user_normalized], axis=1)
# X = np.concatenate([encoded_track_name], axis=1)
# X = np.concatenate(
#     [popularity_normalized, duration_ms_normalized, danceability_normalized, energy_normalized,
#      number_of_matching_genres_normalized], axis=1)

#
# Extract the labels
y = balanced_data['skipped'].astype(int).values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)

# count skipped and not skipped songs in X_train
print("Y_train not skipped", np.count_nonzero(y_test == 0))
print("Y_train skipped", np.count_nonzero(y_test == 1))
print("Y_train skipped %", np.count_nonzero(y_test == 1) / len(y_test))


In [ ]:
def build_model(hp):
  model = Sequential()
  model.add(Dense(3000 * 2, activation='relu', input_shape=(X_train.shape[1],)))
  model.add(Dropout(0.5))
  model.add(Dense(3000, activation='relu'))
  model.add(Dropout(0.5))
  # model.add(Dense(1000, activation='relu'))
  # model.add(Dropout(0.5))
  model.add(Dense(1, activation='sigmoid'))

  # model.compile(
  #     optimizer=keras.optimizers.Adam(hp.Choice('learning_date', values=[0.5, 0.1, 0.01])),
  #     loss='binary_crossentropy', metrics=['accuracy'])
  model.compile(
      optimizer=Adam(),
      loss='binary_crossentropy', metrics=['accuracy'])
  return model


In [ ]:
epochs = 25
batch_size = 64

model = build_model(None)

history = model.fit(
    X_train,
    y_train,
    validation_data=(X_test, y_test),
    epochs=epochs,
    batch_size=batch_size,
    verbose=2
)

# tuner = keras_tuner.tuners.Hyperband(
#     build_model,
#     objective='val_accuracy',
#     max_epochs=50,
#     max_trials=10,
#     executions_per_trial=2,
#     directory='my_dir')


In [ ]:
# tuner.search(
#     (X_train, y_train),
#     validation_data=(X_test, y_test),
# )

In [ ]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test, verbose=2)
print("Test set accuracy:", accuracy)

print("TEST")
y_pred = model.predict(X_test)
y_pred_classes = (y_pred > 0.5).astype(int)
print("Accuracy:", accuracy_score(y_test, y_pred_classes))
print("Confusion matrix:\n", confusion_matrix(y_test, y_pred_classes))
print("Classification report:\n", classification_report(y_test, y_pred_classes))

print("TRAIN")
y_pred = model.predict(X_train)
y_pred_classes = (y_pred > 0.5).astype(int)
print("Accuracy:", accuracy_score(y_train, y_pred_classes))
print("Confusion matrix:\n", confusion_matrix(y_train, y_pred_classes))
print("Classification report:\n", classification_report(y_train, y_pred_classes))

In [ ]:
# # Make predictions on the test set
# y_pred = model.predict(X_test)
# y_pred_classes = (y_pred > 0.5).astype(int)
#
# # Print the predicted and actual labels
# print("Predicted labels:", y_pred_classes.flatten())
# print("Actual labels:", y_test)

In [ ]:
# # test with new data
#
# new_data = [
#   (["dominican pop", "merengue", "merengue tipico", "tropical"],
#    ["blues rock", "country rock", "lounge"])
# ]
#
# labels = [
#
# ]
#
# new_df = pd.DataFrame(new_data, columns=["genres", "favourite_genres"])
#
# # Combine genres and favourite_genres
# all_new_genres = list(new_df['genres'] + new_df['favourite_genres'])
#
# # One-hot encode the genres using the previously fit MultiLabelBinarizer (mlb)
# encoded_new_genres = mlb.transform(all_new_genres)
#
# # Split encoded_new_genres into genres and favourite_genres
# encoded_new_genres1 = encoded_new_genres[:, :len(new_df['genres'][0])]
# encoded_new_genres2 = encoded_new_genres[:, len(new_df['genres'][0]):]
#
# # Concatenate the one-hot encoded columns
# X_new = np.concatenate([encoded_new_genres1, encoded_new_genres2], axis=1)
#
# y_new_pred = model.predict(X_new)
# y_new_pred_classes = (y_new_pred > 0.5).astype(int)
#
# # Print the predicted labels
# print("Predicted labels:", y_new_pred_classes.flatten())
# print("Actual labels:", labels)
